In [12]:
import dspy
from groq import Groq
import os
from dotenv import load_dotenv
import json
load_dotenv('../llama_3_function_calling_groq/.env')

True

In [15]:
lm = dspy.LM('groq/llama3-70b-8192', api_key=os.getenv('GROQ_API_KEY'))
dspy.configure(lm=lm)

## sentiment classification

In [17]:
sentence = "it's a charming and often affecting journey."  # example from the SST-2 dataset.

classify = dspy.Predict('sentence -> sentiment: bool')  # we'll see an example with Literal[] later
classify(sentence=sentence).sentiment

True

## summarization

In [18]:
# Example from the XSum dataset.
document = """The 21-year-old made seven appearances for the Hammers and netted his only goal for them in a Europa League qualification round match against Andorran side FC Lustrains last season. Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's but was unable to save them from relegation. The length of Lee's contract with the promoted Tykes has not been revealed. Find all the latest football transfers on our dedicated page."""

summarize = dspy.ChainOfThought('document -> summary')
response = summarize(document=document)

print(response.summary)

Footballer Lee signs with Barnsley after playing for West Ham and having loan spells.


In [19]:
print("Reasoning:", response.reasoning)

Reasoning: The text is about a 21-year-old footballer named Lee who has signed a contract with Barnsley (the Tykes) after playing for West Ham United and having loan spells with Blackpool and Colchester United.


In [20]:
from typing import Literal

class Emotion(dspy.Signature):
    """Classify emotion."""

    sentence: str = dspy.InputField()
    sentiment: Literal['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'] = dspy.OutputField()

sentence = "i started feeling a little vulnerable when the giant spotlight started blinding me"  # from dair-ai/emotion

classify = dspy.Predict(Emotion)
classify(sentence=sentence)

Prediction(
    sentiment='fear'
)

In [21]:
class CheckCitationFaithfulness(dspy.Signature):
    """Verify that the text is based on the provided context."""

    context: str = dspy.InputField(desc="facts here are assumed to be true")
    text: str = dspy.InputField()
    faithfulness: bool = dspy.OutputField()
    evidence: dict[str, list[str]] = dspy.OutputField(desc="Supporting evidence for claims")

context = "The 21-year-old made seven appearances for the Hammers and netted his only goal for them in a Europa League qualification round match against Andorran side FC Lustrains last season. Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's but was unable to save them from relegation. The length of Lee's contract with the promoted Tykes has not been revealed. Find all the latest football transfers on our dedicated page."

text = "Lee scored 3 goals for Colchester United."

faithfulness = dspy.ChainOfThought(CheckCitationFaithfulness)
faithfulness(context=context, text=text)

Prediction(
    reasoning="The text claims that Lee scored 3 goals for Colchester United, but the context states that he scored twice for the U's.",
    faithfulness=False,
    evidence={"Lee scored twice for the U's": ["He scored twice for the U's but was unable to save them from relegation."]}
)

In [22]:
qa = dspy.Predict('question: str -> response: str')
response = qa(question="what are high memory and low memory on linux?")

print(response.response)

In Linux, "high memory" and "low memory" refer to the way the kernel manages memory.

Low memory refers to the first 896 MB of RAM, which is directly addressable by the kernel. This region is also known as the "lowmem" or "normal zone". The kernel can directly access this memory without any additional processing.

High memory, on the other hand, refers to the memory above 896 MB. This region is also known as the "highmem" or "high zone". The kernel cannot directly access this memory and needs to use temporary mappings to access it. This is because the kernel's own memory space is limited to the low memory region.

It's worth noting that these terms are somewhat outdated, as modern Linux systems can handle much larger amounts of RAM. However, the distinction between high and low memory remains relevant for understanding how the kernel manages memory.


In [23]:
cot = dspy.ChainOfThought('question -> response')
cot(question="should curly braces appear on their own line?")

Prediction(
    reasoning='The question is asking about the formatting of curly braces in the interaction structure.',
    response='No, curly braces should not appear on their own line. They should be used to enclose the values of the input and output fields.'
)

In [27]:
# dspy.inspect_history(n=1)